# Use the lightgbm models to score the 200 candidates #

In [ ]:
local = False
if local:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/'Kaggle Otto Reccommender'/data
  path_to_module = '/content/drive/MyDrive/Kaggle Otto Reccommender/'
else:
  !mkdir /my_mnt_dir
  !google-drive-ocamlfuse /my_mnt_dir
  %cd /my_mnt_dir/'Kaggle Otto Reccommender'/data
  path_to_module = '/my_mnt_dir/Kaggle Otto Reccommender/'

import sys    
sys.path.append(path_to_module)

In [ ]:
!pip install lightgbm
!pip install fastparquet
!pip install polars

In [ ]:
import glob
import numpy as np
import pandas as pd
import gc
from tqdm import tqdm
from copy import deepcopy
import polars as pl

import lightgbm as lgbm
from otto_utils import create_sub

Lightgbm inference functions

In [ ]:
def inference(dtrain_list, recall_type, folds, features):
  for fold in range(0, folds):
    if fold == 0:
      preds = collect_preds(dtrain_list, recall_type, fold, features)
    else:
      preds += collect_preds(dtrain_list, recall_type, fold, features)
  preds = preds / folds

  return preds

def collect_preds(dtrain_list, recall_type, fold, features):
  for i, dtrain_item in enumerate(tqdm(dtrain_list)):
    model_file = f'../models/lgbm_models/{recall_type}_{fold}.lgbm'
    model = lgbm.Booster(model_file=model_file)
    dtrain_item = dtrain_item.loc[:, features]
    print(dtrain_item)
    preds_chunk = model.predict(dtrain_item)
    if i == 0:
      preds = preds_chunk
    else:
      preds = np.append(preds, preds_chunk)

  return preds

def get_top_20_df(preds, training_data, return_all=True):
  predictions = training_data[['session','aid']].copy()
  predictions['preds'] = preds
  predictions.sort_values(by=['session', 'preds'], ascending=[True, False], inplace=True)
  predictions['n'] = predictions.groupby('session').cumcount() + 1
  if return_all:
    return predictions
  else:
    return predictions.loc[predictions['n'] <= 20]

  return submitted_predictions

In [ ]:
path_to_training_data = './test_training_data'
training_data = pl.read_parquet(f'{path_to_training_data}/training_data.parquet')
# Quick fix to a column that was calculated incorrectly...
training_data = training_data.with_column(((pl.col('percent_of_test_weeks_interacted') * 4) / (3)).alias('percent_of_test_weeks_interacted'))
training_data = training_data.to_pandas()

In [ ]:
for column in training_data.columns:
  if training_data[column].dtype in ('Int32', 'float64', 'Int16'):
    training_data[column] = training_data[column].astype('float32')
training_data.replace([np.inf, -np.inf], np.NaN, inplace=True)
training_data.shape

In [ ]:
aids_and_sessions = training_data[['session','aid']]
features = [feature for feature in training_data.columns if feature not in ['session', 'aid']]
dtrain_list = []
sessions = training_data['session'].unique()
session_lists = [np_array.tolist() for np_array in np.array_split(np.array(sessions), 10 ) ]

for session_list in tqdm(session_lists):
  dtrain_list.append(training_data.query(f'session in {session_list}').copy())
  training_data = training_data.loc[~training_data.session.isin(session_list)]

folds=1
del training_data

100%|██████████| 10/10 [04:45<00:00, 28.52s/it]


Clicks booster

In [ ]:
#Clicks
click_preds = inference(dtrain_list, 'clicks', folds=1, features=features)
clicks_df = get_top_20_df(click_preds, aids_and_sessions, return_all=True)
clicks_df['n'] = clicks_df['n'].astype('int16')
subset = clicks_df.loc[:, ['session', 'aid', 'n']]
subset = subset.reset_index(drop=True)
subset.to_feather(f'{path_to_training_data}/lgb_clicks_df.feather')

In [ ]:
#Carts
cart_preds = inference(dtrain_list, 'carts', folds=5, features=features)
carts_df = get_top_20_df(cart_preds, aids_and_sessions, return_all=True)
carts_df['n'] = carts_df['n'].astype('int16')
subset = carts_df.loc[:, ['session', 'aid', 'n']]
subset = subset.reset_index(drop=True)
subset.to_feather(f'{path_to_training_data}/lgb_carts_df.feather')

In [ ]:
#Orders
order_preds = inference(dtrain_list, 'orders', folds=6, features=features)
orders_df = get_top_20_df(order_preds, aids_and_sessions, return_all=True)
orders_df['n'] = orders_df['n'].astype('int16')
subset = orders_df.loc[:, ['session', 'aid', 'n']]
subset = subset.reset_index(drop=True)
subset.to_feather(f'{path_to_training_data}/lgb_orders_df.feather')